In [ ]:
import os
from os.path import exists
import pickle
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import backend as K
from sklearn.model_selection import cross_val_score, KFold

import pickle
import gc
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
print(os.getcwd())
os.chdir("/home/wmbio/WORK/gitworking/DeepDEP/")
print(os.getcwd())

In [ ]:
def load_data(filename):
    data = []
    gene_names = []
#     data_labels = []
    lines = open(filename).readlines()
    sample_names = lines[0].replace('\n', '').split('\t')[1:]
    dx = 1

    for line in tqdm(lines[dx:], leave=True):
        values = line.replace('\n', '').split('\t')
        gene = str.upper(values[0])
        gene_names.append(gene)
        data.append(values[1:])
    data = np.array(data, dtype='float32')
    data = np.transpose(data)

    return data, sample_names, gene_names

def full_model():
    with tf.device('/cpu:0'):
        model_mut = Sequential()
        model_mut.add(Dense(1000, input_dim=premodel_mut[0][0].shape[0], activation=activation_func,
                            weights=premodel_mut[0], trainable=True))
        model_mut.add(Dense(100, input_dim=1000, activation=activation_func, weights=premodel_mut[1],
                            trainable=True))
        model_mut.add(Dense(50, input_dim=100, activation=activation_func, weights=premodel_mut[2],
                            trainable=True))

        model_exp = Sequential()
        model_exp.add(Dense(1000, input_dim=premodel_exp[0][0].shape[0], activation=activation_func,
                            weights=premodel_exp[0], trainable=True))
        model_exp.add(Dense(100, input_dim=1000, activation=activation_func, weights=premodel_exp[1],
                            trainable=True))
        model_exp.add(Dense(50, input_dim=100, activation=activation_func, weights=premodel_exp[2],
                            trainable=True))

        model_cna = Sequential()
        model_cna.add(Dense(1000, input_dim=premodel_cna[0][0].shape[0], activation=activation_func,
                            weights=premodel_cna[0], trainable=True))
        model_cna.add(Dense(100, input_dim=1000, activation=activation_func, weights=premodel_cna[1],
                            trainable=True))
        model_cna.add(Dense(50, input_dim=100, activation=activation_func, weights=premodel_cna[2],
                            trainable=True))

        model_meth = Sequential()
        model_meth.add(Dense(1000, input_dim=premodel_meth[0][0].shape[0], activation=activation_func,
                             weights=premodel_meth[0], trainable=True))
        model_meth.add(Dense(100, input_dim=1000, activation=activation_func, weights=premodel_meth[1],
                             trainable=True))
        model_meth.add(Dense(50, input_dim=100, activation=activation_func, weights=premodel_meth[2],
                             trainable=True))

        # subnetwork of gene fingerprints
        model_gene = Sequential()
        model_gene.add(Dense(1000, input_dim=data_fprint.shape[1], activation=activation_func, kernel_initializer=init,
                             trainable=True))
        model_gene.add(Dense(100, input_dim=1000, activation=activation_func, kernel_initializer=init, trainable=True))
        model_gene.add(Dense(50, input_dim=100, activation=activation_func, kernel_initializer=init, trainable=True))

        conc = Concatenate()([model_mut.output, model_exp.output, 
                              model_cna.output, model_meth.output, model_gene.output])

        model_pre = Dense(dense_layer_dim, input_dim=250, activation=activation_func, kernel_initializer=init,
                              trainable=True)(conc)
        model_pre = Dense(dense_layer_dim, input_dim=dense_layer_dim, activation=activation_func, kernel_initializer=init,
                              trainable=True)(model_pre)
        model_pre = Dense(1, input_dim=dense_layer_dim, activation=activation_func2, kernel_initializer=init,
                              trainable=True)(model_pre)

        model_final = Model([model_mut.input, model_exp.input, model_cna.input, model_meth.input, model_gene.input], model_pre)
        
        return model_final

def exp_mut_cna_model():
    with tf.device('/cpu:0'):
        model_mut = Sequential()
        model_mut.add(Dense(1000, input_dim=premodel_mut[0][0].shape[0], activation=activation_func,
                            weights=premodel_mut[0], trainable=True))
        model_mut.add(Dense(100, input_dim=1000, activation=activation_func, weights=premodel_mut[1],
                            trainable=True))
        model_mut.add(Dense(50, input_dim=100, activation=activation_func, weights=premodel_mut[2],
                            trainable=True))

        model_exp = Sequential()
        model_exp.add(Dense(1000, input_dim=premodel_exp[0][0].shape[0], activation=activation_func,
                            weights=premodel_exp[0], trainable=True))
        model_exp.add(Dense(100, input_dim=1000, activation=activation_func, weights=premodel_exp[1],
                            trainable=True))
        model_exp.add(Dense(50, input_dim=100, activation=activation_func, weights=premodel_exp[2],
                            trainable=True))

        model_cna = Sequential()
        model_cna.add(Dense(1000, input_dim=premodel_cna[0][0].shape[0], activation=activation_func,
                            weights=premodel_cna[0], trainable=True))
        model_cna.add(Dense(100, input_dim=1000, activation=activation_func, weights=premodel_cna[1],
                            trainable=True))
        model_cna.add(Dense(50, input_dim=100, activation=activation_func, weights=premodel_cna[2],
                            trainable=True))

        # subnetwork of gene fingerprints
        model_gene = Sequential()
        model_gene.add(Dense(1000, input_dim=data_fprint.shape[1], activation=activation_func, kernel_initializer=init,
                             trainable=True))
        model_gene.add(Dense(100, input_dim=1000, activation=activation_func, kernel_initializer=init, trainable=True))
        model_gene.add(Dense(50, input_dim=100, activation=activation_func, kernel_initializer=init, trainable=True))

        conc = Concatenate()([model_mut.output, model_exp.output, 
                              model_cna.output, model_gene.output])

        model_pre = Dense(dense_layer_dim, input_dim=200, activation=activation_func, kernel_initializer=init,
                              trainable=True)(conc)
        model_pre = Dense(dense_layer_dim, input_dim=dense_layer_dim, activation=activation_func, kernel_initializer=init,
                              trainable=True)(model_pre)
        model_pre = Dense(1, input_dim=dense_layer_dim, activation=activation_func2, kernel_initializer=init,
                              trainable=True)(model_pre)

        model_final = Model([model_mut.input, model_exp.input, model_cna.input, model_gene.input], model_pre)
        
        return model_final
    
    
def exp_mut_model():
    with tf.device('/cpu:0'):
        model_mut = Sequential()
        model_mut.add(Dense(1000, input_dim=premodel_mut[0][0].shape[0], activation=activation_func,
                            weights=premodel_mut[0], trainable=True))
        model_mut.add(Dense(100, input_dim=1000, activation=activation_func, weights=premodel_mut[1],
                            trainable=True))
        model_mut.add(Dense(50, input_dim=100, activation=activation_func, weights=premodel_mut[2],
                            trainable=True))

        model_exp = Sequential()
        model_exp.add(Dense(1000, input_dim=premodel_exp[0][0].shape[0], activation=activation_func,
                            weights=premodel_exp[0], trainable=True))
        model_exp.add(Dense(100, input_dim=1000, activation=activation_func, weights=premodel_exp[1],
                            trainable=True))
        model_exp.add(Dense(50, input_dim=100, activation=activation_func, weights=premodel_exp[2],
                            trainable=True))

        # subnetwork of gene fingerprints
        model_gene = Sequential()
        model_gene.add(Dense(1000, input_dim=data_fprint.shape[1], activation=activation_func, kernel_initializer=init,
                             trainable=True))
        model_gene.add(Dense(100, input_dim=1000, activation=activation_func, kernel_initializer=init, trainable=True))
        model_gene.add(Dense(50, input_dim=100, activation=activation_func, kernel_initializer=init, trainable=True))

        conc = Concatenate()([model_mut.output, model_exp.output, model_gene.output])

        model_pre = Dense(dense_layer_dim, input_dim=150, activation=activation_func, kernel_initializer=init,
                              trainable=True)(conc)
        model_pre = Dense(dense_layer_dim, input_dim=dense_layer_dim, activation=activation_func, kernel_initializer=init,
                              trainable=True)(model_pre)
        model_pre = Dense(1, input_dim=dense_layer_dim, activation=activation_func2, kernel_initializer=init,
                              trainable=True)(model_pre)

        model_final = Model([model_mut.input, model_exp.input, model_gene.input], model_pre)
        
        return model_final    

def exp_model():
    with tf.device('/cpu:0'):
        model_exp = Sequential()
        model_exp.add(Dense(1000, input_dim=premodel_exp[0][0].shape[0], activation=activation_func,
                            weights=premodel_exp[0], trainable=True))
        model_exp.add(Dense(100, input_dim=1000, activation=activation_func, weights=premodel_exp[1],
                            trainable=True))
        model_exp.add(Dense(50, input_dim=100, activation=activation_func, weights=premodel_exp[2],
                            trainable=True))

        # subnetwork of gene fingerprints
        model_gene = Sequential()
        model_gene.add(Dense(1000, input_dim=data_fprint.shape[1], activation=activation_func, kernel_initializer=init,
                             trainable=True))
        model_gene.add(Dense(100, input_dim=1000, activation=activation_func, kernel_initializer=init, trainable=True))
        model_gene.add(Dense(50, input_dim=100, activation=activation_func, kernel_initializer=init, trainable=True))

        conc = Concatenate()([model_exp.output, model_gene.output])

        model_pre = Dense(dense_layer_dim, input_dim=100, activation=activation_func, kernel_initializer=init,
                              trainable=True)(conc)
        model_pre = Dense(dense_layer_dim, input_dim=dense_layer_dim, activation=activation_func, kernel_initializer=init,
                              trainable=True)(model_pre)
        model_pre = Dense(1, input_dim=dense_layer_dim, activation=activation_func2, kernel_initializer=init,
                              trainable=True)(model_pre)

        model_final = Model([model_exp.input, model_gene.input], model_pre)
        
        return model_final  
    
def mut_model():
    with tf.device('/cpu:0'):
        model_mut = Sequential()
        model_mut.add(Dense(1000, input_dim=premodel_mut[0][0].shape[0], activation=activation_func,
                            weights=premodel_mut[0], trainable=True))
        model_mut.add(Dense(100, input_dim=1000, activation=activation_func, weights=premodel_mut[1],
                            trainable=True))
        model_mut.add(Dense(50, input_dim=100, activation=activation_func, weights=premodel_mut[2],
                            trainable=True))

        # subnetwork of gene fingerprints
        model_gene = Sequential()
        model_gene.add(Dense(1000, input_dim=data_fprint.shape[1], activation=activation_func, kernel_initializer=init,
                             trainable=True))
        model_gene.add(Dense(100, input_dim=1000, activation=activation_func, kernel_initializer=init, trainable=True))
        model_gene.add(Dense(50, input_dim=100, activation=activation_func, kernel_initializer=init, trainable=True))

        conc = Concatenate()([model_mut.output, model_gene.output])

        model_pre = Dense(dense_layer_dim, input_dim=100, activation=activation_func, kernel_initializer=init,
                              trainable=True)(conc)
        model_pre = Dense(dense_layer_dim, input_dim=dense_layer_dim, activation=activation_func, kernel_initializer=init,
                              trainable=True)(model_pre)
        model_pre = Dense(1, input_dim=dense_layer_dim, activation=activation_func2, kernel_initializer=init,
                              trainable=True)(model_pre)

        model_final = Model([model_mut.input, model_gene.input], model_pre)
        
        return model_final  

class generator(tf.keras.utils.Sequence):
    def __init__(self, x_mut, x_exp, x_cna, x_meth, x_fprint, y_dep, batch_size):
        self.x_mut, self.x_exp, self.x_cna, self.x_meth, self.x_fprint, self.y_dep = x_mut, x_exp, x_cna, x_meth, x_fprint, y_dep
        self.bs = batch_size
    
    def __len__(self):
        return (len(self.y_dep) - 1) // self.bs + 1
    
    def __getitem__(self,idx):
        start, end = idx * self.bs, (idx+1) * self.bs
        return (self.x_mut[start:end], self.x_exp[start:end], self.x_cna[start:end], self.x_meth[start:end], self.x_fprint[start:end]), self.y_dep[start:end]

class generator2(generator):
    def __init__(self, x_mut, x_exp, x_fprint, y_dep, batch_size):
        self.x_mut, self.x_exp, self.x_fprint, self.y_dep = x_mut, x_exp, x_fprint, y_dep
        self.bs = batch_size
        
    def __getitem__(self, idx):
        start, end = idx * self.bs, (idx+1) * self.bs
        return (self.x_mut[start:end], self.x_exp[start:end], self.x_fprint[start:end]), self.y_dep[start:end]

class generator1(generator):
    def __init__(self, x_mut, x_fprint, y_dep, batch_size):
        self.x_mut, self.x_fprint, self.y_dep = x_mut, x_fprint, y_dep
        self.bs = batch_size
        
    def __getitem__(self, idx):
        start, end = idx * self.bs, (idx+1) * self.bs
        return (self.x_mut[start:end], self.x_fprint[start:end]), self.y_dep[start:end]
    
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
    
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def model_train_vis(history):
    coeff = history.history['coeff_determination']
    val_coeff = history.history['val_coeff_determination']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(coeff))
    
    plt.plot(epochs, coeff, 'bo', label='Training')
    plt.plot(epochs, val_coeff, 'b', label='Validation')
    plt.title('Training and validation Coeff determination(R-squared)')
    plt.legend()

    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training')
    plt.plot(epochs, val_loss, 'b', label='Validation')
    plt.title('Training and validation loss(MSE)')
    plt.legend()

    plt.show()


```
# with open('prediction/data/ccl_complete_data_28CCL_1298DepOI_36344samples_demo.pickle', 'rb') as f:
# with open('data/ccl_complete_data_278CCL_1298DepOI_360844samples.pickle', 'rb') as f:
#     data_mut, data_exp, data_cna, data_meth, data_dep, data_fprint = pickle.load(f)
# This pickle file is for DEMO ONLY (containing 28 CCLs x 1298 DepOIs = 36344 samples)!
# First 1298 samples correspond to 1298 DepOIs of the first CCL, and so on.
# For the complete data used in the paper (278 CCLs x 1298 DepOIs = 360844 samples),
# please substitute by 'ccl_complete_data_278CCL_1298DepOI_360844samples.pickle',
# to which a link can be found in README.md
```

## **Data Load & Path**

In [ ]:
TRAIN_PATH = "/home/wmbio/WORK/gitworking/DeepDEP/preprocessing/DATA/2022-07-13/"
TEMP_PATH = "prediction/train_preprocessing/"
SAVE_PATH = "prediction/custom_model/"

In [ ]:
if exists("prediction/data/ccl_complete_data_501CCL_1298DepOI_614727samples_custom.npz") is False:
    # load TCGA mutation data, substitute here with other genomics
    data_exp, sample_names_exp, property_names_exp = load_data(TRAIN_PATH + "train_exp_training.txt")
    if exists(TEMP_PATH + "train_exp_training.npy") is False:
        np.save(TEMP_PATH + 'train_exp_training.npy', data_exp)

    data_mut, sample_names_mut, property_names_mut = load_data(TRAIN_PATH + "train_mut_training.txt")
    if exists(TEMP_PATH + "train_mut_training.npy") is False:
        np.save(TEMP_PATH + 'train_mut_training.npy', data_mut)

    data_cna, sample_names_cna, property_names_cna = load_data(TRAIN_PATH + "train_cna_training.txt")
    if exists(TEMP_PATH + "train_cna_training.npy") is False:
        np.save(TEMP_PATH + 'train_cna_training.npy', data_cna)

    data_meth, sample_names_meth, property_names_meth = load_data(TRAIN_PATH + "train_meth_training.txt")
    if exists(TEMP_PATH + "train_meth_training.npy") is False:
        np.save(TEMP_PATH + 'train_meth_training.npy', data_meth)

    data_dep, sample_names_dep, property_names_dep = load_data(TRAIN_PATH + "train_DepScore_training.txt")
    if exists(TEMP_PATH + "train_DepScore_training.npy") is False:
        np.save(TEMP_PATH + 'train_DepScore_training.npy', data_dep)

    data_fprint, sample_names_fprint, property_names_fprint = load_data(TRAIN_PATH + "train_fingerprint_training.txt")
    if exists(TEMP_PATH + "train_fingerprint_training.npy") is False:
        np.save(TEMP_PATH + 'train_fingerprint_training.npy', data_fprint)

    np.savez_compressed("prediction/data/ccl_complete_data_501CCL_1298DepOI_614727samples_custom.npz", 
                        data_exp=data_exp, data_mut=data_mut, data_cna=data_cna, data_meth=data_meth,
                        data_dep=data_dep, data_fprint=data_fprint)
else :
    data = np.load("prediction/data/ccl_complete_data_501CCL_1298DepOI_614727samples_custom.npz")
    data_exp = data['data_exp']
    data_mut = data['data_mut']
    data_cna = data['data_cna']
    data_meth = data['data_meth']
    data_fprint = data['data_fprint']
    data_dep = data['data_dep']

## **Build Model**

* **Pretrained Model load - TCGA**

In [ ]:
# Load autoencoders of each genomics that were pre-trained using 8238 TCGA samples
# New autoencoders can be pretrained using PretrainAE.py
premodel_mut = pickle.load(open(SAVE_PATH + 'pretrained/premodel_tcga_custom_mut_1000_100_50.pickle', 'rb'))
premodel_exp = pickle.load(open(SAVE_PATH + 'pretrained/premodel_tcga_custom_exp_1000_100_50.pickle', 'rb'))
premodel_cna = pickle.load(open(SAVE_PATH + 'pretrained/premodel_tcga_custom_cna_1000_100_50.pickle', 'rb'))
premodel_meth = pickle.load(open(SAVE_PATH + 'pretrained/premodel_tcga_custom_meth_1000_100_50.pickle', 'rb'))

In [ ]:
activation_func = 'relu' # for all middle layers
activation_func2 = 'linear' # for output layer to output unbounded gene-effect scores
init = 'he_uniform'
dense_layer_dim = 250
batch_size = 64
num_epoch = 100
num_DepOI = 1227 # 1298 DepOIs as defined in our paper, custom 1227
num_ccl = int(data_mut.shape[0]/num_DepOI)
split_ratio = 0.8

In [ ]:
# 80% CCLs for training/validation, and 20% for testing
id_rand = np.random.permutation(num_ccl)
id_cell_train = id_rand[np.arange(0, round(num_ccl * split_ratio))]
id_cell_test = id_rand[np.arange(round(num_ccl * split_ratio), num_ccl)]
id_train = np.arange(0, num_DepOI) + id_cell_train[0]*num_DepOI
for y in id_cell_train:
    id_train = np.union1d(id_train, np.arange(0, num_DepOI) + y*num_DepOI)
id_test = np.arange(0, num_DepOI) + id_cell_test[0] * num_DepOI
for y in id_cell_test:
    id_test = np.union1d(id_test, np.arange(0, num_DepOI) + y*num_DepOI)
print("\n\nTraining/validation on %d samples (%d CCLs x %d DepOIs) and testing on %d samples (%d CCLs x %d DepOIs).\n\n" % (
    len(id_train), len(id_cell_train), num_DepOI, len(id_test), len(id_cell_test), num_DepOI))

In [ ]:
# tf.keras.utils.plot_model(model_final, show_shapes=True,  show_dtype=True, to_file='model_custom.png')

## **Fit & Evaluation Model**

### **(1) Full Model 4-Omics**

In [ ]:
train_generator = generator(data_mut[id_train], data_exp[id_train], data_cna[id_train], data_meth[id_train], 
                                    data_fprint[id_train], data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator(data_mut[id_test], data_exp[id_test], data_cna[id_test], data_meth[id_test], 
                                    data_fprint[id_test], data_dep[id_test], batch_size = batch_size)

In [ ]:
model = full_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
SAVE_NAME = "model_custom_full_0720"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

In [ ]:
model_train_vis(history=history)

### **(2) Expression-Mutation-CNA Model**

In [ ]:
train_generator = generator(data_mut[id_train], data_exp[id_train], data_cna[id_train], 
                                    data_fprint[id_train], data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator(data_mut[id_test], data_exp[id_test], data_cna[id_test],
                                    data_fprint[id_test], data_dep[id_test], batch_size = batch_size)

In [ ]:
model = full_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
SAVE_NAME = "model_custom_mut_exp_cna_0725"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(3) Expression-Mutation Model**

In [ ]:
train_generator = generator2(data_mut[id_train], data_exp[id_train], data_fprint[id_train],
                            data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator2(data_mut[id_test], data_exp[id_test], data_fprint[id_test],
                           data_dep[id_test], batch_size = batch_size)

In [ ]:
model = exp_mut_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
model_train_vis(history=history)

In [ ]:
SAVE_NAME = "model_custom_mut_exp_0720"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(3) Expression Model**

In [ ]:
train_generator = generator1(data_exp[id_train], data_fprint[id_train],
                            data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator1(data_exp[id_test], data_fprint[id_test],
                           data_dep[id_test], batch_size = batch_size)

In [ ]:
model = exp_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
model_train_vis(history=history)

In [ ]:
SAVE_NAME = "model_custom_exp_0720"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(4) Mutation Model**

In [ ]:
train_generator = generator1(data_mut[id_train], data_fprint[id_train],
                            data_dep[id_train], batch_size = batch_size)

In [ ]:
test_generator = generator1(data_mut[id_test], data_fprint[id_test],
                           data_dep[id_test], batch_size = batch_size)

In [ ]:
model = mut_model()
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='min')
model.compile(loss='mse', optimizer='adam', metrics=[coeff_determination])
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=num_epoch, verbose=1,         
                    callbacks = [es])

In [ ]:
model_train_vis(history=history)

In [ ]:
SAVE_NAME = "model_custom_mut_0720"
model.save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))